In [1]:
import torch, torchvision
from segmentation_models.segmentation_models_pytorch.decoders.fpn import model
import segmentation_models_pytorch as smp
#from tests import test_loaded_weights, compare_output
%load_ext autoreload
%autoreload 2
#torch.backends.cudnn.deterministic = True

/home/windows/anaconda3/envs/vision/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
class FPNclassifier(smp.FPN):
    def __init__(self,backbone = 'timm-res2net50_14w_8s', n_classes=2, encoder_weights=None):
        # https://github.com/qubvel/segmentation_models.pytorch#encoders 
        assert backbone in ['timm-res2net50_14w_8s','efficientnet-b7', 'timm-mobilenetv3_large_minimal_100',
                            'timm-mobilenetv3_small_100']
        activation = 'softmax' if n_classes > 2 else 'sigmoid'
        super().__init__(backbone, in_channels=3, encoder_depth=5, decoder_merge_policy='cat', encoder_weights=encoder_weights,
                        aux_params={'classes':n_classes, 'pooling': "avg", 'activation':activation})
        
        activation = 'softmax' if n_classes > 2 else 'sigmoid'
        
    def forward(self, x):
        x = self.encoder(x)[-1]
        class_out = torch.nn.Sequential(*list(model.classification_head.children())[:1])(x)
        return class_out
    
     
class Linknetclassifier(smp.Linknet):
    def __init__(self,backbone = 'timm-res2net50_14w_8s', n_classes=2, encoder_weights=None):
        # https://github.com/qubvel/segmentation_models.pytorch#encoders 
        assert backbone in ['timm-res2net50_14w_8s','efficientnet-b7', 'timm-mobilenetv3_large_minimal_100',
                            'timm-mobilenetv3_small_100']
        activation = 'softmax' if n_classes > 2 else 'sigmoid'
        super().__init__(backbone, in_channels=3, encoder_depth=5, encoder_weights=encoder_weights,
                        aux_params={'classes':n_classes, 'pooling': "avg", 'activation':activation})
        
        activation = 'softmax' if n_classes > 2 else 'sigmoid'
        
    def forward(self, x):
        x = self.encoder(x)[-1]
        class_out = self.classification_head(x)
        return class_out

In [26]:
model = FPNclassifier()

In [27]:
x = torch.randn((3,3,1024,1024))
model(x).size()

torch.Size([3, 2048, 1, 1])